In [ ]:
%pip install speechrecognition gTTS langchain langchain-google-genai google-generativeai pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for pyau

In [ ]:
%pip install PyAudio‑0.2.11‑cp311‑cp311‑win_amd64.whl

ERROR: PyAudio‑0.2.11‑cp311‑cp311‑win_amd64.whl is not a valid wheel filename.


In [ ]:
GOOGLE_API_KEY = "your-google-api-key-here"

In [ ]:
%pip install SpeechRecognition

In [ ]:
import os
import speech_recognition as sr
from gtts import gTTS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Set your Gemini API key
os.environ["GOOGLE_API_KEY"] = "your-google-api-key-here"

# Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    max_tokens=1024,
    max_retries=2,
)

# Function to generate interview questions
def generate_interview_questions(job_role, job_description, interview_type):
    prompt_template = """
You are an expert interviewer. Based on the following job role, job description,
and interview type, generate 5 tailored interview questions that assess a candidate's fit for the role.

Job Role: {job_role}
Job Description: {job_description}
Interview Type: {interview_type}

Provide the questions as a numbered list.
"""
    prompt = PromptTemplate(
        input_variables=["job_role", "job_description", "interview_type"],
        template=prompt_template
    )
    full_prompt = prompt.format(
        job_role=job_role,
        job_description=job_description,
        interview_type=interview_type
    )
    response = llm.invoke(full_prompt)
    questions = [line.strip() for line in response.content.split("\n") if line.strip() and line.strip()[0].isdigit()]
    return questions

# Function to record audio and transcribe
def record_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Speak now...")
        audio = r.listen(source, timeout=5, phrase_time_limit=10)
    try:
        text = r.recognize_google(audio)
        print(f"Transcribed: {text}")
        return text
    except Exception as e:
        return f"Error recognizing speech: {e}"

# Function to generate feedback
def generate_feedback(transcript):
    feedback_prompt = """
You are an expert interviewer. Please review the following interview transcript and provide detailed feedback including:
- Strengths in the candidate's responses,
- Areas for improvement,
- Overall suggestions for interview improvement,
- An overall score out of 10.

Interview Transcript:
{transcript}

Provide your response in a clear, structured format.
"""
    prompt = PromptTemplate(
        input_variables=["transcript"],
        template=feedback_prompt
    )
    full_prompt = prompt.format(transcript=transcript)
    response = llm.invoke(full_prompt)
    return response.content

# Main flow
if __name__ == "__main__":
    role = input("Enter job role: ")
    description = input("Enter job description: ")
    interview_type = input("Enter interview type (e.g., behavioral, technical): ")

    questions = generate_interview_questions(role, description, interview_type)
    print("\n Interview Questions:")
    for q in questions:
        print(q)

    print("\n Answer one of the questions by speaking...")
    transcript = record_audio()

    print("\n Generating feedback...")
    feedback = generate_feedback(transcript)
    print("\n Interview Feedback:\n")
    print(feedback)

    # Optional: Convert feedback to speech
    tts = gTTS(text=feedback, lang='en')
    tts.save("feedback.mp3")
    print("Feedback saved as 'feedback.mp3'")

Enter job description: Cooking
